In [ ]:
!pip install ray==2.41.0

In [ ]:
!pip install numpy mlflow tensorflow "ray[serve,default,client]"

In [ ]:
import mlflow
import mlflow.tensorflow
import numpy as np

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# -------------------
# Prepare Data
# -------------------
data = load_diabetes()
X = data.data
y = data.target.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -------------------
# Define Model
# -------------------
def create_model(input_dim):
    model = keras.Sequential([
        layers.Dense(64, activation="relu", input_shape=(input_dim,)),
        layers.Dense(32, activation="relu"),
        layers.Dense(1)  # regression output
    ])
    model.compile(optimizer="adam", loss="mse", metrics=["mse"])
    return model

input_dim = X_train.shape[1]
epochs = 50
batch_size = 32

mlflow.set_experiment("Diabetes_Prediction_TensorFlow")

with mlflow.start_run():
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("optimizer", "adam")
    mlflow.log_param("loss_fn", "mse")
    mlflow.log_param("input_features", input_dim)

    model = create_model(input_dim)

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0
    )

    # Evaluation
    loss, mse = model.evaluate(X_test, y_test, verbose=0)
    rmse = np.sqrt(mse)

    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)

In [ ]:
import numpy as np
import mlflow.tensorflow
import tensorflow as tf
from starlette.requests import Request
from typing import Dict

from ray import serve
import ray


@serve.deployment(
    ray_actor_options={
        "runtime_env": {
            "pip": ["tensorflow"]
        },
    }
)
class TensorFlowMLflowDeployment:
    def __init__(self):
        print("Loading model...")
        self.model = model
        print("Model loaded successfully.")

    async def __call__(self, input_data) -> Dict:
        try:
            if isinstance(input_data, Request):
                data = await input_data.json()
            else:
                data = input_data
            features = data.get("features", None)
            if features is None:
                return {"error": "Missing 'features' in request"}
            X = np.array(features).reshape(1, -1)

            # Make prediction with TensorFlow model
            prediction = self.model.predict(X).flatten().tolist()

            return {"prediction": prediction}
        except Exception as e:
            return {"error": str(e)}


# Bind and deploy
app = TensorFlowMLflowDeployment.bind()
handle = serve.run(app, route_prefix="/predict")

In [ ]:
json_data = {"features": [0.0380759, 0.0506801, 0.0616962, 0.0218724, -0.0442235, -0.0348208, -0.0434008, -0.00259226, 0.0199084, -0.0176461]}
response = handle.remote(json_data)
await response